# (18) df: shatter

**Motivation**: Device = ```cuda:0``` <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from analysis.eval import sparse_score
from figures.fighelper import *
from vae.train_vae import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Fig save dir

In [2]:
from figures.imgs import plot_weights

fig_dir = pjoin(fig_base_dir, 'nips_may22')
os.makedirs(fig_dir, exist_ok=True)
print(os.listdir(fig_dir))

kws_fig = {
    'transparent': True,
    'bbox_inches': 'tight',
    'dpi': 600,
}

['nelbo_point.pdf']

## Device

In [3]:
device_idx = 0
device = f'cuda:{device_idx}'

from analysis.final import sort_fits, analyze_fits

## Load, make df

### Fit names

In [4]:
fits, fits_st, fits_etc = sort_fits()
len(fits), len(fits_st), len(fits_etc)

(460, 90, 527)

### df

In [5]:
def accept_fn(name):
    archi = name.split('-')[1]
    accept = (
        'MNIST' in name and
        archi == '<conv+b|conv+b>'
    )
    return accept


fits = sorted(filter(accept_fn, fits))
fits_etc = sorted(filter(accept_fn, fits_etc))

print(f"# fits: {len(fits)},  # fits_etc: {len(fits_etc)}")

# fits: 100,  # fits_etc: 90

In [6]:
%%time

kws = dict(device=device, analysis_mode='shatter', shatter_clf='logreg')
df = pd.concat([analyze_fits(fits, **kws), analyze_fits(fits_etc, **kws)])

df = df.sort_values(by=['type', 'seed', 'n_dims'])
df = df.reset_index(drop=True)

save_obj(
    obj=df,
    file_name='df_shatter',
    save_dir=tmp_dir,
    verbose=True,
    mode='df',
)

100%|███████████████████████████████████████████| 90/90 [20:49<00:00, 13.89s/it]


[PROGRESS] 'df_shatter.df' saved at
/home/hadi/Dropbox/git/jb-vae/tmp

CPU times: user 18h 48min 5s, sys: 1h 37min 18s, total: 20h 25min 23s
Wall time: 51min


In [8]:
df['type'].value_counts()

type
categorical          6300
gaussian             6300
poisson              6300
laplace              6300
gaussian-relu        3780
gaussian-exp         3780
gaussian-square      2520
gaussian-softplus    2520
laplace-exp          2520
laplace-relu         2520
laplace-softplus     2520
laplace-square       2520
Name: count, dtype: int64

In [10]:
df.pivot_table(
    values='accuracy',
    index=['type'],
    columns=['n_dims'],
    aggfunc=np.mean,
)

n_dims,5,10,20,50,100
type,,,,,
categorical,0.729934,0.795400,0.869850,0.898817,0.883566
gaussian,0.734849,0.757773,0.742540,0.743305,0.763185
gaussian-exp,NaN,0.736986,NaN,0.761785,0.793041
gaussian-relu,NaN,0.762299,NaN,0.786000,0.817990
gaussian-softplus,NaN,0.751143,NaN,0.761620,NaN
gaussian-square,NaN,0.793973,NaN,0.808752,NaN
laplace,0.731724,0.751364,0.753212,0.770346,0.767038
laplace-exp,NaN,0.716556,NaN,0.741154,NaN
laplace-relu,NaN,0.770200,NaN,0.811704,NaN
